In [58]:
# Importing Neccesary Libaries
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.ensemble import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.neural_network import MLPClassifier

In [59]:
# Reading Processed Train Set Data File
raw_d1 = pd.read_csv('Data/Processed_Trainset.csv')

# Feature Headers List
headers = list(raw_d1.columns)

# Reading Test Data Set
raw_d2 = pd.read_csv('Data/Processed_Testset.csv')

id_col = list(raw_d2.Client_ID)
raw_d2.drop(['Client_ID'], axis=1, inplace=True)

In [60]:
# Generating Correlation Matrix
X_T = np.array(raw_d1)[:,:-1]
corr_mat = pd.DataFrame(np.corrcoef(X_T.T), columns=headers[:-1], index=headers[:-1])
corr_mat.to_csv('Data/Correlation_Matrix.csv')

In [61]:
# Convert Data into ndArrays
train_data = np.array(raw_d1)
test_data = np.array(raw_d2)

# Total Training Data
X_tot = train_data[:, :-1]
Y_tot = train_data[:, -1]

# Splitting Data into a Training Set and a Cross Validation Set
X_train, X_cross, Y_train, Y_cross = train_test_split(X_tot, Y_tot, test_size = 0.2, random_state = 42, shuffle = True)

In [62]:
# # Standard Normalizing Data
sc = StandardScaler()
sc.fit(X_tot)
X_tot = sc.fit_transform(X_tot)
X_train = sc.transform(X_train)
X_cross = sc.transform(X_cross)
test_data = sc.fit_transform(test_data)

In [63]:
# Applying the Classification Model - Random Forest
# rf = RandomForestClassifier(bootstrap=True, class_weight= None, max_depth= 8, max_features= 10, max_leaf_nodes= None,
#                             min_samples_leaf= 1, min_samples_split= 2, min_weight_fraction_leaf= 0.0, n_estimators= 100,
#                             n_jobs= 1, oob_score= False, random_state= 85, verbose= 0, warm_start= False)

rf = XGBClassifier()

# rf = GradientBoostingClassifier()
# rf = MLPClassifier(hidden_layer_sizes=(20, 20), activation='tanh', solver='adam', alpha=0.0001, batch_size='auto', 
#                    learning_rate='constant', learning_rate_init=0.1, power_t=0.5, max_iter=200, shuffle=True, 
#                    random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, 
#                    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
#                    n_iter_no_change=10, max_fun=15000)

rf.fit(X_train, Y_train)

# Predicting Outcomes
predict_train = rf.predict(X_train)
predict_cross = rf.predict(X_cross)
predict_tot = rf.predict(X_tot)
predict_test = rf.predict(test_data)

In [64]:
# Computing Confusion Matrix on Cross Validation Set
confusion = confusion_matrix(Y_cross,predict_cross)

# Computing Accuracy Values
accuracy_train = accuracy_score(Y_train, predict_train)
accuracy_cross = accuracy_score(Y_cross, predict_cross)
accuracy_tot = accuracy_score(Y_tot, predict_tot)

# Computing AUC Values
auc_train = roc_auc_score(Y_train, predict_train)
auc_cross = roc_auc_score(Y_cross, predict_cross)
auc_tot = roc_auc_score(Y_tot, predict_tot)                

# Printing Accuracy Metrics
print("Train Accuracy            : {:.3f}%       AUC: {:.4f}".format(accuracy_train * 100, auc_train))
print("Cross Validation Accuracy : {:.3f}%       AUC: {:.4f} ".format(accuracy_cross * 100, auc_cross))
print("Total Set Accuracy        : {:.3f}%       AUC: {:.4f}".format(accuracy_tot * 100, auc_tot))
print("Confusion Matrix for Cross Validation Set :\n %s" % str(confusion))
print("Number of Ones Predicted for Test Data : %d" % np.sum(predict_test))

print(classification_report(Y_cross, predict_cross))

Train Accuracy            : 82.365%       AUC: 0.6612
Cross Validation Accuracy : 82.667%       AUC: 0.6612 
Total Set Accuracy        : 82.425%       AUC: 0.6612
Confusion Matrix for Cross Validation Set :
 [[3582  163]
 [ 669  386]]
Number of Ones Predicted for Test Data : 654
              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90      3745
         1.0       0.70      0.37      0.48      1055

    accuracy                           0.83      4800
   macro avg       0.77      0.66      0.69      4800
weighted avg       0.81      0.83      0.80      4800



In [65]:
# Uploading Predictions to a CSV
df = pd.DataFrame(predict_test, dtype=int)
df.index = id_col
df.to_csv('Data/Submission.csv', header=['NEXT_MONTH_DEFAULT'], index_label='Client_ID')